In [1]:
import os
import openai
from pprint import pprint
import os 

openai.api_key = os.getenv("OPENAPI_KEY")

In [4]:
class TransformBuilder():
    def __init__(self, layers, example_input, example_output):
        self.layers = layers if layers else 10
        self.example_input = example_input
        self.example_output = example_output
        self.message_log = [{
                "role": "system", 
                "content": ''' 
                    You are an AI which writes python scripts to extract transform and load input data into output data. 
                    I will give you an example of input data and output data. 
                    You will give me only the code written as a function called data_transform
                '''
            }]

        
    def take_step(self, message):
        print(self.message_log[-1]['content'])
        self.message_log.append({
                "role": "user", 
                "content": message
            })
    
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # The name of the OpenAI chatbot model to use
            messages=self.message_log,   # The conversation history up to this point, as a list of dictionaries
            max_tokens=1000,        # The maximum number of tokens (words or subwords) in the generated response
            stop=None,              # The stopping sequence for the generated response, if any (not used here)
            temperature=0.7,        # The "creativity" of the generated response (higher temperature = more creative)
        )

        self.message_log.append({"role": "assistant", "content": response.choices[0].message.content})

    def generate_transform_function(self):

        # Make an initial message with the input and output
        message = f'''
            Input = {str(input_data)}
            Ouput = {str(output_data)}
        '''

        self.message_log = self.take_step(message)
        correct = False
        i = 0
        while correct == False and  i < self.layers:
            try: 
                # Run the suggested code
                code = self.message_log[-1]['content'].split("```")[1].replace("python", '')
                exec(code)
                new_output = data_transform(input_data)

                if new_output != output_data:
                    message = f'''
                        The script you wrote for me did not give the correct output, 
                        please re-write the script so it has the following output:
                        {output_data}
                    '''
                    self.message_log = self.take_step(message)
                    i += 1 
                else:
                    correct = True

                    #TODO - Have the transform builder set its own transform() funtion 
                    self.data_transform = data_transform

                    i += 1 

            except Exception as e:
                message = f'''
                    The script you wrote for me gave me an error when I ran it, the error it gave is:
                    {str(e)}
                    re-write the script with the error fixed
                '''
                self.message_log = self.take_step(message)
                i += 1 

In [5]:
input_data = {
    'firstname' : 'Hayden', 
    'lastname' : 'Salmon',
    'address' : "3227 Gardner Road, Lakeland Florida 33810"
}

output_data = {
    'firstName' : "Hayden", 
    'lastName' : "Salmon",
    'fullname' : "Hayden Salmon",
    'address' : {
        "street" : "3227 Gardner Road",
        "city" : "Lakeland",
        "state" : "Florida",
        "zip" : "33810"
    }
}


model = TransformBuilder(10, input_data, output_data)
model.generate_transform_function()

 
                    You are an AI which writes python scripts to extract transform and load input data into output data. 
                    I will give you an example of input data and output data. 
                    You will give me only the code written as a function called data_transform
                


TypeError: 'NoneType' object is not subscriptable